# Code minimal nécessaire pour Query

In [1]:
from typing import Callable, List
import re, os

BASE_DIR = os.getcwd()
DATA_FOLDER = os.path.join(BASE_DIR, "data", "BULLETINS")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "output")
INDEX_OUTPUT_DIR = os.path.join(OUTPUT_FOLDER, "index_files")

XML_INITIAL_FILE = os.path.join(OUTPUT_FOLDER, "corpus_initial.xml")
ANTI_DICT_FILE = os.path.join(OUTPUT_FOLDER, "anti_dictionnaire.txt")
STEMMED_REPLACEMENTS = os.path.join(OUTPUT_FOLDER, "stemmed_replacement.tsv")
LEMMATIZED_REPLACEMENTS = os.path.join(OUTPUT_FOLDER, "lemmatized_replacement.tsv")

TEXTE_INDEX_FILE = os.path.join(INDEX_OUTPUT_DIR, "texte_index.xml")
LEGENDE_INDEX_FILE = os.path.join(INDEX_OUTPUT_DIR, "legende_index.xml")
TITRE_INDEX_FILE = os.path.join(INDEX_OUTPUT_DIR, "titre_index.xml")

STORAGE_TAGS = {"Corpus": "corpus", "documents": "bulletins", "Document": "bulletin", "Image": "image"}

# Fonction de standardisation
STANDARDIZE: Callable[[str], str] = lambda x: re.sub(r"[^\w\s]", "", re.sub(r"'", " ", x.strip().lower()))

## Récupération du Corpus (valeurs de résultat, documents à retrouver)

In [2]:
from index import Corpus

with open(XML_INITIAL_FILE, "r", encoding="utf-8") as f:
    CORPUS = Corpus.model_validate_xml(f.read(), tags=STORAGE_TAGS)

## Récupération de l'Index des documents

In [3]:
from index import InvertedIndex
import pandas

INDEX = {}

for zone, file in zip(["texte", "legendes", "titre"], [TEXTE_INDEX_FILE, LEGENDE_INDEX_FILE, TITRE_INDEX_FILE]):
    INDEX[zone] = InvertedIndex.from_dataframe(pandas.read_csv(file, sep="\t", encoding="utf-8"))
    
list(INDEX["texte"].keys())[:3]

C:\Users\ongki\AppData\Local\Temp\ipykernel_10348\835930859.py:7: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257

['le', '27', 'avril']

## Standardisation Complète

In [4]:
substitutions_df = pandas.read_csv(LEMMATIZED_REPLACEMENTS, sep="\t", header=None, na_filter=False)
substitutions = {inp: repl for inp, repl in zip(substitutions_df.iloc[:,0], substitutions_df.iloc[:,1])}

TOKENIZE: Callable[[List[str]], List[str]] = lambda x: [substitutions.get(STANDARDIZE(token), STANDARDIZE(token)) for token in x]

TOKENIZE(re.findall(r"\w+", "Bonjour abondante j'aime le canapé"))

['bonjour', 'abondant', 'j', 'aime', 'le', 'canapé']

In [5]:
from index import spacy_lemmatize

def TOKENIZE_SAVE(x):
    for token in x:
        token = STANDARDIZE(token)
        if token in substitutions:
            yield substitutions[token]
        else:
            new_lemma = spacy_lemmatize(token)[0]
            substitutions[token] = new_lemma
            with open(STEMMED_REPLACEMENTS, "a", encoding="utf-8") as f:
                f.write(f"{token}\t{new_lemma}\n")
            yield new_lemma
                
list(TOKENIZE_SAVE(re.findall(r"\w+", "Bonjour abondante j'aime le canapé")))

['bonjour', 'abondant', 'j', 'aime', 'le', 'canaper']

## Querying

In [6]:
from index import Query
q = Query.build(
    'montre moi les articles qui parlent de "environnement", minimum depuis juillet 2011 et qui nest pas dans la rubrique Focus',
)
for field in ["keywords", "excluded_keywords", "title_contains"]:
    setattr(q, field, TOKENIZE(getattr(q, field)))
for field in ["rubriques", "excluded_rubriques"]:
    setattr(q, field, [STANDARDIZE(r) for r in getattr(q, field)])
q

Query(keywords=['environnement'], keywords_operator='AND', rubriques=[], excluded_keywords=[], excluded_rubriques=['focus'], date_start=datetime.datetime(2011, 7, 1, 0, 0, tzinfo=TzInfo(UTC)), date_end=None, excluded_date_periods=[], title_contains=[], has_image=False, target_info='articles')

In [7]:
results = q.search(documents=CORPUS, index=INDEX, debug=True)
results

Removed 69813.htm rubrique is excluded
Removed 68881.htm rubrique is excluded
Removed 72393.htm rubrique is excluded
Removed 70421.htm rubrique is excluded
Removed 72116.htm rubrique is excluded
Removed 72392.htm rubrique is excluded
Removed 67071.htm too old
Removed 73875.htm rubrique is excluded
Removed 67938.htm rubrique is excluded
Removed 68275.htm rubrique is excluded
Removed 69811.htm rubrique is excluded
Removed 69533.htm rubrique is excluded
Removed 73877.htm rubrique is excluded
Removed 69535.htm rubrique is excluded
Removed 69177.htm rubrique is excluded
Removed 73431.htm rubrique is excluded
Removed 75064.htm rubrique is excluded
Removed 75458.htm rubrique is excluded
Removed 70161.htm rubrique is excluded
Removed 69812.htm rubrique is excluded
Removed 73684.htm rubrique is excluded
Removed 70420.htm rubrique is excluded
Removed 70915.htm rubrique is excluded
Removed 73432.htm rubrique is excluded
Removed 67794.htm rubrique is excluded
Removed 71357.htm rubrique is excluded

[Document(fichier='70746.htm', numero='272', date=datetime.datetime(2012, 8, 9, 0, 0), rubrique='En direct des laboratoires', titre='Des bactéries flagellées capables de torpiller des biofilms', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='INRA/Jouy - Romain Briandet - email : romain.briandet@jouy.inra.fr', texte="Pour la plupart, les microorganismes vivent fixés entre eux et, de préférence, à une surface, formant ainsi ce que l'on appelle des biofilms qui peuplent notre environnement quotidien (hôpitaux, jardins, salles de bains, réfrigérateurs ...). La cohésion de ces assemblages tridimensionnels que sont ces biofilms est assurée par une matrice organique constituée d'eau et de biopolymères autoproduits qui lui confèrent ses propriétés visco-élastiques. Il faut savoir que les cellules d'un biofilm peuvent être 1.000 fois plus tolérantes à l'action d'un agent antimicrobien que leurs homologues dispersées dans un liquide. Or l'organisation spatiale d'un biofi

In [8]:
for result in results:
    print(result.rubrique)

En direct des laboratoires
A lire
Evénement
Actualité Innovation
Evénement
Actualité Innovation
En direct des labos
En direct des laboratoires
Evénement
Horizon Enseignement
Actualité Innovation
Evénement
A lire
Horizons Enseignement
Actualité Innovation
Au coeur des régions
Actualité Innovation
En direct des laboratoires
En direct des laboratoires
Du côté des pôles
Au coeur des régions
Du côté des pôles
En direct des laboratoires
Evénement
Horizons Formation Enseignement
Au coeur des régions
Actualité Innovation
Au coeur des régions
En direct des laboratoires
Horizons Enseignement
Actualité Innovation
Evénement
Au coeur des régions
Actualité Innovation
Evénement
A lire
Actualité Innovation
Evénement
Evénement
Horizon Enseignement
Evénement
Au coeur des régions
Du côté des Pôles
Actualité Innovation
Actualité Innovation
Au coeur des régions
Du côté des pôles
Actualité-Innovation
Evénement
En direct des laboratoires
Actualités Innovations
Au coeur des régions
